# Propp Wilson Algorithm for the Ferromagnetic Ising Model #

<b>1.</b> Ferromagnetic Model:

<b>2.</b> Ising Model:

<b>Functions of the Ising Model:</b>
* `runIsing(t, state, U):`

* `updateState(t, state, U):`

* `calcEnergyDiff(i, j, state):`

* `InitialState(N):`

### Declaration of the libraries used in the program

In [1]:
using Pkg; Pkg.activate("."); Pkg.instantiate()

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


In [2]:
using Distributions, LinearAlgebra
using Plots; pyplot(); default(legendfontsize = 15.0, linewidth = 2.0)
using Random

### Utility Functions

In [3]:
function InitialState(N)  # Generates an initial state with all values fixed to +1 and -1 ---OK
    state = zeros(3, N, N)
    for i in 1:N
        for j in 1:N
            state[1, i, j] = 1
            state[2, i, j] = -1
        end
    end
    return state
end

InitialState (generic function with 1 method)

* Calculate the energy difference

In [4]:
function calcEnergyDiff(i, j, state)  # Calculate the energy at flipping the vertex at [i,j]
    m = size(state, 2)
    if i == 1
        top = 1
    else
        top = state[i - 1, j]
    end
    if i == m
        bottom = 1
    else
        bottom = state[i + 1, j]
    end
    if j == 1
        left = 1
    else
        left = state[i, j - 1]
    end
            
    if j == m
        right = 1
    else
        right = state[i, j + 1]
    end

    energy = 2 * state[i,j] * sum([top, bottom, left, right])  # Energy calculated by given formula
                
    return energy
end
    


calcEnergyDiff (generic function with 1 method)

* Update State for Ising Model

In [5]:
function updateState(t, state, U,T)
    B = 1 / T
    E = zeros(3)

    i = Int(U[2,Int(-t)])  # Picks a random vertex, the same each time the chain runs from 0
    j = Int(U[3,Int(-t)])
    
    for h in 1:3

        E[h] =  calcEnergyDiff(i, j, state[h,:,:])  # Find energy under randomly generated flip of each state space separately

        u = U[1,Int(-t)]
        if state[h, i, j] == 1
            u = 1 - u
        end
        
        if u < 0.5 * (1 - tanh(0.5 * B * E[h]))  # condition to accept change, random number is the same each time
            state[h, i, j] = -state[h, i, j]
            #n[h]=1
        else
            state[h, i, j] = state[h, i, j]
            #n[h]=0
        end
    end

    return state  # returns both states
end

updateState (generic function with 1 method)

### Ising Model Function

In [6]:
function runIsing(t, state, U, T)  # Runs chain from the designated starting time until time 0

    while t < 0
        state = updateState(t, state, U,T)
        t += 1
    end
    return state
end

runIsing (generic function with 1 method)

## Propp Wilson Algorithm

<b>Introduction to Propp Wilson Algorithm</b>

The propp-Wilson Algorithm solves two different problems
* s' is distributed exactly π (" perfect simulation ")
* The algorithm <b>stops</b> automatically

### Pseudocode of the algorithm:
<b>1.</b> Set 𝑚 = 1

<b>2.</b> For each 𝑠 ∈ 𝑆, simulate the Markov chain starting at time −𝑁𝑚 in state 𝑠 

and running up to time 0 using the update function 𝜙 with the sequence 𝑈−𝑁𝑚 +1,𝑈−𝑁𝑚 +2, … ,𝑈−1,𝑈0

<b>3.</b> If all 𝑘 chains end up in the same state s at time 0, output s and stop

<b>4.</b> Set 𝑚: = 𝑚 + 1 and go to step 2

<b>Note:</b> The same sequence 𝑈0,𝑈−1, … is used for all 𝑘 chains.


<b>Functions in Propp Wilson Algorithm</b>
* runProppWilson(N,j):
* genRandomness(N,M): 
* genStartingTimes(j):

``genRandomness(N,M)``: generates a sequence of i.i.d. random numbers for Propp-Wilson Algorithm
````

U0,U1,U2...

````

In [7]:
function genRandomness(N, M)  # generate and store three sets of random numbers
    U = zeros(3, M)

    U[2,:] = rand(1:N, M) # Random numbners i
    U[3,:] = rand(1:N, M)  # Random numbners j
    for i in 1:M
        U[1,i] = rand()  # Random numbners U
    end
    return U
end

genRandomness (generic function with 1 method)

* Generate starting times

``genStartingTimes(j)``: to generate the starting times, the function generates the sequence of the positive numbers like (N1,N2...) = (1,2,4,8)

In [8]:
function genStartingTimes(j)  # Creates starting times, each one is double the previous
    M = zeros(j)
    M[2] = 1
    for x in 3:j
        M[x] = 2 * M[x - 1]
    end       
    return M
end

genStartingTimes (generic function with 1 method)


* Propp Wilson Algorithm call the Ising model to run chain from the designated starting time until time 0 after each magnetization
 

In [9]:
function runProppWilson(N,j,T)
    M = genStartingTimes(j)# negative integers (-1,-2,-4,-8)
    U = genRandomness(N, 1)# U0,U1,U2...
    state = InitialState(N)
    m=2
    while state[1,:,:] != state[2,:,:]  # Condition for termination: both state spaces are the same
        U = hcat(U,genRandomness(N, Int(M[m] - M[m-1]))) # Generates more random numbers when necessary
        
        magnetization = sum([sum(i) for i in state[1,:,:]])-sum([sum(i) for i in state[2,:,:]])
        
        println("magnetization= ",magnetization, "round= ", m)
        state = runIsing(-Int(M[m]), state,U, T)
        m += 1  # If states are not the same, goes to the next starting time
        
    end
    return state[1,:,:]
end

runProppWilson (generic function with 1 method)

## Graph ##

* Main Program  and Visualisation of the Result
* This Graph function below call the ProppWilson Algorithm.


In [10]:
function visualizeMagnetization(magnetization)
    step = 1:30;
    plot(magnetization,step )
end

visualizeMagnetization (generic function with 1 method)

In [13]:
state = runProppWilson(10, 20,2)

magnetization= 200.0round= 2
magnetization= 200.0round= 3
magnetization= 200.0round= 4
magnetization= 200.0round= 5
magnetization= 200.0round= 6
magnetization= 200.0round= 7
magnetization= 198.0round= 8
magnetization= 194.0round= 9
magnetization= 190.0round= 10
magnetization= 176.0round= 11
magnetization= 160.0round= 12
magnetization= 138.0round= 13
magnetization= 60.0round= 14


10×10 Array{Float64,2}:
 1.0  1.0  -1.0  1.0   1.0  1.0  1.0  1.0  1.0   1.0
 1.0  1.0   1.0  1.0   1.0  1.0  1.0  1.0  1.0   1.0
 1.0  1.0   1.0  1.0   1.0  1.0  1.0  1.0  1.0   1.0
 1.0  1.0  -1.0  1.0   1.0  1.0  1.0  1.0  1.0   1.0
 1.0  1.0   1.0  1.0   1.0  1.0  1.0  1.0  1.0   1.0
 1.0  1.0   1.0  1.0   1.0  1.0  1.0  1.0  1.0   1.0
 1.0  1.0   1.0  1.0   1.0  1.0  1.0  1.0  1.0   1.0
 1.0  1.0   1.0  1.0   1.0  1.0  1.0  1.0  1.0   1.0
 1.0  1.0   1.0  1.0  -1.0  1.0  1.0  1.0  1.0   1.0
 1.0  1.0   1.0  1.0   1.0  1.0  1.0  1.0  1.0  -1.0

In [11]:
function Graph(N, j, T)
    state = runProppWilson(N, j,T)
    S = size(state)[2]
    for i in 1:S
        for j in 1:S
            if state[i,j] == 1  # Graphs a red + if the matrix entry is positive
                println("+1")
            elseif state[i,j] == -1# blue - if the matrix entry is negative
                println("-1")
            end
        end
    end
end


Graph (generic function with 1 method)

* Show Results
Run the program with the parameter below.

In [12]:
T = 2
Graph(10,20,T)

magnetization= 200.0round= 2
magnetization= 200.0round= 3
magnetization= 200.0round= 4
magnetization= 198.0round= 5
magnetization= 198.0round= 6
magnetization= 198.0round= 7
magnetization= 196.0round= 8
magnetization= 188.0round= 9
magnetization= 176.0round= 10
magnetization= 144.0round= 11
magnetization= 122.0round= 12
magnetization= 86.0round= 13
magnetization= 44.0round= 14
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
-1
-1
+1
+1
+1
+1
+1
+1
+1
+1
-1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
-1
+1
+1
+1
